**Introduction**

The purpose of this notebook is to evaluate my fine-tuned bert-base-uncased model on questions that do not belong in the SQuAD 2.0 dataset.
For this purpose, i will use some articles from wikipedia of my choice and questions i make myself about each of these articles. I will also test it with a paragraph i wrote myself.

**Mount Google Drive(to read the model's parameters)**

First of all, we must mount the google drive storage to load the parameters of the fine-tuned models. I did that this way because the files are too big to upload multiple times.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Install pytorch interface for pre-trained BERT**

Then, we must install the interface to use the pre-trained BERT models.

In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 12.4 MB/s 
     |████████████████████████████████| 895 kB 36.5 MB/s 
     |████████████████████████████████| 596 kB 47.4 MB/s 
     |████████████████████████████████| 6.5 MB 45.9 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


**Import necessary libraries**

Now, it's time to import the necessary libraries for this notebook.

In [3]:
import torch
import re
import string
import collections
from transformers import BertTokenizerFast, BertForQuestionAnswering

**Enable CUDA**

Enable CUDA for GPU utilization by our model. This makes calculations faster.

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

**BERT model definition**

And here, we define the name of the pre-trained bert model from [Hugging Face](https://huggingface.co/models) which we used previously on the fine-tuning process.

In [5]:
BERT_MODEL_NAME = 'bert-base-uncased'

**Download BERT tokenizer**

In [6]:
tokenizer = BertTokenizerFast.from_pretrained(BERT_MODEL_NAME)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

**Evaluation script**

The script used to calculate f1 and exact match scores between gold and predicted answers. This is the same script from the SQuAD website which is defined [here](https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/).

In [7]:
def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
  return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def predict(model: BertForQuestionAnswering, query: str, context: str):
  with torch.no_grad():
    model.eval()
    inputs = tokenizer.encode_plus(text=context, text_pair=query, max_length=512, padding='max_length', truncation=True, return_tensors='pt').to(device)
    outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], token_type_ids=inputs['token_type_ids'])
    ans_start = torch.argmax(outputs[0])
    ans_end = torch.argmax(outputs[1])
    ans = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][ans_start:ans_end+1]))
    return ans

**Load pre-trined SQuAD model**

Now, we will load the BERT model and the parameters from the pre training we did previously.

In [9]:
model = BertForQuestionAnswering.from_pretrained(BERT_MODEL_NAME).to(device)
model.load_state_dict(torch.load('/content/gdrive/MyDrive/NLP exercises/Question Answering/Model Parameters/model_squad.pt'))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

<All keys matched successfully>

**Question Answering time**

Now the time all of you were waiting for....

Time to try some question-answering with data not in SQuAD 2.0 dataset!!!

Source: [https://en.wikipedia.org/wiki/PlayStation_Portable](https://en.wikipedia.org/wiki/PlayStation_Portable)

In [78]:
context = "The PlayStation Portable[a] (PSP) is a handheld game console developed and marketed by Sony Computer Entertainment. It was first released in Japan on December 12, 2004, in North America on March 24, 2005, and in PAL regions on September 1, 2005, and is the first handheld installment in the PlayStation line of consoles. As a seventh generation console, the PSP competed with the Nintendo DS."

In [82]:
query = "What is PSP?"
predict(model, query, context)

'a handheld game console'

In [79]:
query = "What console generation was the PSP?"
predict(model, query, context)

'seventh'

In [83]:
query = "Which company was the PSP developed by?"
predict(model, query, context)

'sony computer entertainment'

In [80]:
query = "When was the PSP released?"
predict(model, query, context)

'december 12, 2004'

In [81]:
query = "Who was a competitor of the PSP?"
predict(model, query, context)

'nintendo ds'

Source: [https://en.wikipedia.org/wiki/Kamatero](https://en.wikipedia.org/wiki/Kamatero)

In [25]:
context = "Kamatero (Greek: Καματερó [kamateˈro]; officially Καματερόν[2]) is a suburb northwest of Athens city center, Greece. Since the 2011 local government reform it is part of the municipality Agioi Anargyroi-Kamatero, of which it is a municipal unit.[3] It is located about 8 kilometres north of Athens city centre."

In [28]:
query = "What is Kamatero?"
predict(model, query, context)

'a suburb northwest of athens city center, greece'

In [30]:
query = "Where is Kamatero located?"
predict(model, query, context)

'about 8 kilometres north of athens city centre'

In [31]:
query = "Which municipality did Kamatero became part of?"
predict(model, query, context)

'agioi anargyroi'

Source (translated from greek): [https://el.wikipedia.org/wiki/%CE%A0%CE%B1%CE%B3%CE%BA%CF%81%CE%AC%CF%84%CE%B9_%CE%91%CF%87%CE%B1%CE%90%CE%B1%CF%82](https://el.wikipedia.org/wiki/%CE%A0%CE%B1%CE%B3%CE%BA%CF%81%CE%AC%CF%84%CE%B9_%CE%91%CF%87%CE%B1%CE%90%CE%B1%CF%82)

In [14]:
context = "Pagrati Achaia is a village in the Prefecture of Achaia located in the southeast of the Prefecture and is 192 km from Athens, 112 km from Patras, 15 km from Klitoria, 37.5 km from Kalavrita and 52 km from Tripoli. It is also located near the famous Cave of the Lakes (in Kastria, Achaia) and very close to Klima tou Pausanias [1] (in Sella, Kalavrita)."

In [15]:
query = "What is Pagrati Achaia?"
predict(model, query, context)

'a village in the prefecture of achaia located in the southeast of the prefecture'

In [16]:
query = "Where is Pagrati?"
predict(model, query, context)

'southeast'

In [17]:
query = "What attraction is Pagrati near to?"
predict(model, query, context)

'cave of the lakes'

In [18]:
query = "What attraction is Pagrati close to?"
predict(model, query, context)

''

In [19]:
query = "How far is Pagrati from Athens?"
predict(model, query, context)

'192 km'

In [20]:
query = "Where is Sella?"
predict(model, query, context)

'kalavrita'

Source (translated from Greek): [https://el.wikipedia.org/wiki/%CE%9C%CE%B7%CF%84%CF%81%CE%BF%CF%80%CE%BF%CE%BB%CE%B9%CF%84%CE%B9%CE%BA%CF%8C_%CE%A0%CE%AC%CF%81%CE%BA%CE%BF_%C2%AB%CE%91%CE%BD%CF%84%CF%8E%CE%BD%CE%B7%CF%82_%CE%A4%CF%81%CE%AF%CF%84%CF%83%CE%B7%CF%82%C2%BB](https://el.wikipedia.org/wiki/%CE%9C%CE%B7%CF%84%CF%81%CE%BF%CF%80%CE%BF%CE%BB%CE%B9%CF%84%CE%B9%CE%BA%CF%8C_%CE%A0%CE%AC%CF%81%CE%BA%CE%BF_%C2%AB%CE%91%CE%BD%CF%84%CF%8E%CE%BD%CE%B7%CF%82_%CE%A4%CF%81%CE%AF%CF%84%CF%83%CE%B7%CF%82%C2%BB)

In [21]:
context = "The Metropolitan Park \"Antonis Tritsis\" (official name: Metropolitan Park of Environmental and Educational Activities and Development of Social Economy \"Antonis Tritsis\"), also known as the Environmental Awareness Park \"Antonis Tritsis is a large public park and is a large parks of Greece. It is located within the geographical boundaries of the municipality of Ilio. The final name of the park was given in honor of the former mayor of Athens, Antonis Tritsis. During the years of liberation, many Turkish estates in Attica were sold by wealthy Greeks and foreign landowners, the most famous of whom was Ioannis Papatheodorou Lefakis who came from Andros. The estate of Lefakis, whose area is about 300 acres, was located at the site where today is the Queen's Tower in the homonymous estate. Ioannis Lefakis, after the formalization of the property titles, sold it in May 1838 for the price of 10,000 drachmas, to the Englishmen John Williams and George Miles, who were willing to invest in the state. By their own order, vineyards, fruit trees were planted, facilities were set up, gardens were set up and arable land was cultivated. In 1840, they dissolved co-ownership and George Miles sold his stake in John Williams. George Miles, who could not cultivate on his own, paid for gardeners from Malta and farmers from Menidi to come and help him. In 1848, John Williams decided to leave Greece and so the property was sold to Dimitrios Kontakis, at a price of 10,370 drachmas."

In [22]:
query = "Where is the Metropolitan Park Antonis Tritsis located?"
predict(model, query, context)

'ilio'

In [23]:
query = "In honor of who was the final name of the park given of?"
predict(model, query, context)

'antonis tritsis'

In [24]:
query = "Who was Ioannis Lefakis?"
predict(model, query, context)

'wealthy greeks and foreign landowners'

Source: [https://el.wikipedia.org/wiki/%CE%A3%CF%84%CF%81%CE%B1%CF%80%CE%B1%CF%84%CF%83%CE%AC%CE%B4%CE%B1](https://el.wikipedia.org/wiki/%CE%A3%CF%84%CF%81%CE%B1%CF%80%CE%B1%CF%84%CF%83%CE%AC%CE%B4%CE%B1)

In [35]:
context = "Strapatsada (Greek: στραπατσάδα) is a popular dish in many regions of Greece, due to the availability and low cost of its ingredients (fresh tomatoes, eggs and olive oil). It is often prepared \"on the spot\" and served for lunch or a light snack; however, it can also be served cold. The dish is also known as kagianas,[1][2] koskosela (in Cyclades) or menemeni. Preparing strapatsada is quick and easy: the chopped or pureed tomatoes are cooked in a frying pan with olive oil and pepper until they become a thick sauce. The beaten eggs are then added and stirred to a boil. Feta cheese can optionally be added just before turning off the heat (salt is usually not necessary if feta is used). Oregano, thyme or other dried herbs can be used as seasoning.[3]The dish is especially popular in the summer, when fresh tomatoes abound. The name comes from the Italian word \"strapazzare\" which means breaking an egg and mix it all while cooking."

In [36]:
query = "What is Strapatsada?"
predict(model, query, context)

'fresh tomatoes, eggs and olive oil'

In [37]:
query = "What is Strapatsada made of?"
predict(model, query, context)

'fresh tomatoes, eggs and olive oil'

In [40]:
query = "What is the region of Strapatsada?"
predict(model, query, context)

'greece'

In [38]:
query = "How is Strapatsada cooked?"
predict(model, query, context)

'in a frying pan'

Source: [https://en.wikipedia.org/wiki/Eridu](https://en.wikipedia.org/wiki/Eridu)

In [41]:
context = "Eridu (Sumerian: 𒉣𒆠, NUN.KI/eridugki; Akkadian: irîtu; modern Arabic: Tell Abu Shahrain) is an archaeological site in southern Mesopotamia (modern Dhi Qar Governorate, Iraq). Eridu was long considered the earliest city in southern Mesopotamia.[1] Located 12 km southwest of Ur, Eridu was the southernmost of a conglomeration of Sumerian cities that grew around temples, almost in sight of one another. These buildings were made of mud brick and built on top of one another.[2] With the temples growing upward and the village growing outward, a larger city was built.[2] In Sumerian mythology, Eridu was originally the home of Enki, later known by the Akkadians as Ea, who was considered to have founded the city. His temple was called E-Abzu, as Enki was believed to live in Abzu, an aquifer from which all life was believed to stem."

In [43]:
query = "What is Eridu?"
predict(model, query, context)

'an archaeological site in southern mesopotamia'

In [42]:
query = "Where is Eridu?"
predict(model, query, context)

'southern mesopotamia'

In [44]:
query = "Who was the founder of Eridu?"
predict(model, query, context)

'enki'

In [45]:
query = "Who was Enki?"
predict(model, query, context)

'ea'

In [46]:
query = "Who was EA?"
predict(model, query, context)

'enki'

In [50]:
query = "What buildings did Eridu have?"
predict(model, query, context)

'temples'

In [51]:
query = "How were Eridu's temples built?"
predict(model, query, context)

'on top of one another'

In [52]:
query = "With what materials were Eridu's temples built?"
predict(model, query, context)

'mud brick'

Source (translated from Greek): [https://el.wikipedia.org/wiki/%CE%A0%CE%BB%CE%B1%CF%84%CE%B1%CE%BD%CE%B9%CF%83%CF%84%CF%8C%CF%82_%CE%95%CF%8D%CE%B2%CE%BF%CE%B9%CE%B1%CF%82](https://el.wikipedia.org/wiki/%CE%A0%CE%BB%CE%B1%CF%84%CE%B1%CE%BD%CE%B9%CF%83%CF%84%CF%8C%CF%82_%CE%95%CF%8D%CE%B2%CE%BF%CE%B9%CE%B1%CF%82)

In [53]:
context = "Platanistos is a semi-mountainous settlement of the Municipality of Karystos in the Regional Unit of Evia. It has a population of 74 inhabitants, according to the 2011 census, and is located 22 km east of Karystos and 140 km southeast of Chalkida. It is the southernmost and closest village of Cabo Doro and has evolved into a modern settlement, with a regional clinic.Its sights are the river, with the vegetation of plane trees (which also gave its name), the stone bridge, the area Anemopyles etc.The Local Community of Platanistos also consists of the settlements of Agioi, Eleonas, Livadi, Kastri, Mourtia, Mastrogiannaika, Epanochori, Pidoulaika and Potami, with a total population of 188 inhabitants."

In [54]:
query = "What is Platanistos?"
predict(model, query, context)

'a semi - mountainous settlement'

In [55]:
query = "Where is Platanistos?"
predict(model, query, context)

'22 km east of karystos and 140 km southeast of chalkida'

In [56]:
query = "What is the population of Platanistos?"
predict(model, query, context)

'74'

In [58]:
query = "What are some attractions Platanistos?"
predict(model, query, context)

'the river, with the vegetation of plane trees'

In [59]:
query = "Does Platanistos have beach nearby?"
predict(model, query, context)

'it is the southernmost and closest village of cabo doro and has evolved into a modern settlement, with a regional clinic. its sights are the river, with the vegetation of plane trees'

In [60]:
query = "What does the local community of Platanistos consist of?"
predict(model, query, context)

'agioi, eleonas, livadi, kastri, mourtia, mastrogiannaika, epanochori, pidoulaika and potami'

And now a brief about me written by me!!!


In [121]:
context = "Vasilis Kiriakopoulos is a 22 year old young man born at Marousi, Athens and currently living at Kamatero, Athens. He doesn't have any brothers or sisters, but has a pet canary named Socrates. He is an undergraduate student at the Department of Informatics and Telecommunications of the National and Kapodistrian University of Athens. He has passion about technology, maths, algorithms, and computers and he also likes problem solving a lot. He is currently doing an internship at Netcompany-Intrasoft and he expects to receive his BSc degree on June 2022. His favourite activities are bike riding, video games and feeding the ducks at the park."

In [122]:
query = "Who is Vasilis Kiriakopoulos?"
predict(model, query, context)

'a 22 year old young man'

In [123]:
query = "Where was Vasilis Kiriakopoulos born?"
predict(model, query, context)

'marousi, athens'

In [124]:
query = "Where does Vasilis Kiriakopoulos currently live?"
predict(model, query, context)

'kamatero, athens'

In [125]:
query = "Does Vasilis Kiriakopoulos have any brothers or sisters?"
predict(model, query, context)

"doesn't have any brothers or sisters"

In [126]:
query = "What pet does Vasilis Kiriakopoulos currently have?"
predict(model, query, context)

'canary named socrates'

In [127]:
query = "What is Vasilis's canary name?"
predict(model, query, context)

'socrates'

In [129]:
query = "Where does Vasilis Kiriakopoulos study?"
predict(model, query, context)

'department of informatics and telecommunications'

In [130]:
query = "What university does Vasilis Kiriakopoulos study at?"
predict(model, query, context)

'kapodistrian university of athens'

In [131]:
query = "What is Vasilis Kiriakopoulos's passion?"
predict(model, query, context)

'technology, maths, algorithms, and computers'

In [138]:
query = "Where does Vasilis Kiriakopoulos doing his internship?"
predict(model, query, context)

'netcompany - intrasoft'

In [133]:
query = "When is Vasilis Kiriakopoulos expected to graduate?"
predict(model, query, context)

'june 2022'

In [134]:
query = "What are Vasilis Kiriakopoulos's favourite activities?"
predict(model, query, context)

'bike riding, video games and feeding the ducks at the park'

As you can see, my model generalizes nicely even on data that it probably hasn't seen during training which is nice!!!